<a href="https://colab.research.google.com/github/danrosher/colab/blob/main/FineTuneSemanticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -qq install numpy datasets sentence_transformers transformers pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.7 MB/s eta 0:

In [ ]:

import numpy as np
from datasets import load_dataset

from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

from random import sample, seed, shuffle
from sentence_transformers import InputExample, losses, evaluation
from torch.utils.data import DataLoader
import pandas as pd

KeyboardInterrupt: ignored

In [ ]:
sbert_model = SentenceTransformer('msmarco-distilbert-base-v4')
#sbert_model = SentenceTransformer('multi-qa-mpnet-base-cos-v1')
sbert_model.max_seq_length = 256



In [ ]:
from google.colab import drive

drive.mount('/g')

In [ ]:
df = pd.read_csv('/g/MyDrive/semantic_search/data.csv')


In [ ]:
df.head()

In [ ]:
def convert_data(df):
  return list(df.apply(lambda x: InputExample(texts = (x['k'], x['description'][:800]),label = float(x['label'])), axis = 1)) 

In [ ]:
good_training_data = df[df['label']==1].sample(n=2000)
bad_training_data = df[df['label']==0].sample(n=2000)

In [ ]:
#training_data = df.sample(frac=1) #return all in random order
sampled_training_data = convert_data(pd.concat([good_training_data,bad_training_data]).sample(frac=1))
len(sampled_training_data)

In [ ]:
sampled_training_data[0].__dict__

In [ ]:
# Define the train dataset, the dataloader and the train loss
train_dataloader = DataLoader(sampled_training_data, shuffle=True, batch_size=64)

train_loss = losses.CosineSimilarityLoss(sbert_model)

In [ ]:
training_index = int(.8 * len(sampled_training_data))  # Get an 80/20 train/test split
train_examples = sampled_training_data[:training_index]


In [ ]:
# Evaluation data
evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(sampled_training_data[training_index:], main_similarity=evaluation.SimilarityFunction.COSINE)
sbert_model.evaluate(evaluator)

In [ ]:
# Tune the model
sbert_model.fit(
    train_objectives=[(train_dataloader, train_loss)], 
    output_path='/g/MyDrive/semantic_search/model',
    save_best_model=True,
    epochs=10,
#     warmup_steps=len(sampled_training_data) // 5, 
    evaluator=evaluator, 
    evaluation_steps=100
)

In [ ]:
sbert_model.save('/g/MyDrive/semantic_search/results')


In [ ]:
evaluator(sbert_model,output_path='/g/MyDrive/semantic_search')